In [2]:
import requests as req
import pandas as pd
from bs4 import BeautifulSoup as bs

In [3]:
hockey_teams = ['WSH', 'PIT', 'CAR', 'NYI', 'CBJ', 'NJD', 
                'PHI', 'NYR', 'BUF', 'BOS', 'TOR', 'MTL', 
               'TBL', 'FLA', 'DET', 'OTT', 'COL', 'NSH',
               'STL', 'WPG', 'DAL', 'CHI', 'MIN', 'EDM',
               'ANA', 'VEG', 'ARI', 'CGY', 'VAN', 'SJS',
               'LAK']

base_url = "https://www.hockey-reference.com/teams/{}/{}.html"

data = req.get("https://www.hockey-reference.com/teams/CHI/2019.html")

In [4]:
stat_id = "skaters"
df_full = pd.DataFrame()

# loop through each hockey team and get data
for team in hockey_teams:
    for year in range(2020, 2021):
        url = base_url.format(team, year)
        data = req.get(url)
        soup = bs(data.text, "html.parser")

        statTable = soup.find(id = stat_id)

        # if no data, then will let us know
        if not statTable:
            print("Error: {}, {}".format(team, year))
    
        rows = statTable.find_all('tr') 
        listed = []
        for row in rows[:-1]:
            cols = row.find_all('td')
            cols = [x.text.strip() for x in cols]  # strip white space

            # if empty cols
            if cols:
                # appending year and team name data to row
                cols += [year, team]
                listed.append(cols)

        # get headers of table
        headers = []
        head_cols = statTable.find('thead').find_all('tr')[1].find_all('th')

        act_headers = []
        abr_headers = []

        for header in head_cols[1:]:
            act_header = header.get('aria-label').strip()
            abr_header = header.text.strip()
            act_headers.append(act_header)
            abr_headers.append(abr_header)
        
        # appending year and team name to the header
        act_headers += ["Year", "Team"]

        # putting the header before the actual data
        listed = [act_headers]+listed

        df = pd.DataFrame(listed)
        df_full = df_full.append(df, ignore_index = True)

In [5]:
df_full.head()

,0,1,2,3,4,5,6,7,8,9,...,20,21,22,23,24,25,26,27,28,29
0,Player,Age,Pos,Games Played,Goals,Assists,Points,Plus/Minus,Penalties in Minutes,Even Strength Goals,...,Offensive Point Shares,Defensive Point Shares,Point Shares,Blocks,Hits,Faceoff Wins,Faceoff Losses,Faceoff Percentage,Year,Team
1,John Carlson,30,D,9,3,14,17,4,4,2,...,1.7,0.8,2.5,19,4,0,0,,2020,WSH
2,Alex Ovechkin,34,LW,9,5,4,9,-2,2,3,...,0.9,0.2,1.0,3,16,0,0,,2020,WSH
3,T.J. Oshie,33,C,9,6,2,8,0,0,3,...,0.9,0.2,1.1,4,10,15,14,51.7,2020,WSH
4,Nicklas Backstrom,32,C,9,2,6,8,-3,2,2,...,0.5,0.1,0.6,3,1,71,71,50.0,2020,WSH
